# Signature Verification

**Problem Statement -** Forgery at banks has become very common and a serious crime. Here we present a deep learning model to tackle this rising problem

**What have we used??**
- We have used *Convolutional Neural Networks* to extract inherent features from a sample signature.
- For loss function we have used *Triplet Loss Function* 
- Model can be trained using *One Shot Learning* but for better results were much better with *Few shot Learning*

### **Step1:** Import Dependcies

In [2]:
## Keras have been used for simlicity of models
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first') ## setting image format -- 

Using TensorFlow backend.


In [3]:
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

### Step2: Using Pre-Trained Inception Model

In [4]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [5]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


### Step3: Defining Loss function

<img src="triplet_loss.jpg">

**Triplet Loss Function:** As the name suggests we calculate difference of positive distance and negative distance. Positive Distance is calculated using anchor and positive while Negative Distance is calculated using anchor and negative image.

In [6]:
def triplet_loss(y_true, y_pred, alpha = 0.2):

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))
    # distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))

    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)

    loss = tf.maximum(basic_loss,0)
    return loss

In [7]:
## Testing triplet loss function
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 350.02716


### Step3: Compiling Model & Loading Weights

In [8]:
def load_weights_from_FaceNet(FRmodel):
    ## WEIGHTS - list of all the weights names
    weights = WEIGHTS
    weights_dict = load_weights()

    # Set layer weights of the model
    for name in weights:
        if FRmodel.get_layer(name) != None:
            FRmodel.get_layer(name).set_weights(weights_dict[name])
        elif model.get_layer(name) != None:
            model.get_layer(name).set_weights(weights_dict[name])

def load_weights():
    # Set weights path
    dirPath = './weights'
    fileNames = filter(lambda f: not f.startswith('.'), os.listdir(dirPath))
    paths = {}
    weights_dict = {}

    for n in fileNames:
        paths[n.replace('.csv', '')] = dirPath + '/' + n

    for name in WEIGHTS:
        if 'conv' in name:
            conv_w = genfromtxt(paths[name + '_w'], delimiter=',', dtype=None)
            conv_w = np.reshape(conv_w, conv_shape[name])
            conv_w = np.transpose(conv_w, (2, 3, 1, 0))
            conv_b = genfromtxt(paths[name + '_b'], delimiter=',', dtype=None)
            weights_dict[name] = [conv_w, conv_b]     
        elif 'bn' in name:
            bn_w = genfromtxt(paths[name + '_w'], delimiter=',', dtype=None)
            bn_b = genfromtxt(paths[name + '_b'], delimiter=',', dtype=None)
            bn_m = genfromtxt(paths[name + '_m'], delimiter=',', dtype=None)
            bn_v = genfromtxt(paths[name + '_v'], delimiter=',', dtype=None)
            weights_dict[name] = [bn_w, bn_b, bn_m, bn_v]
        elif 'dense' in name:
            dense_w = genfromtxt(dirPath+'/dense_w.csv', delimiter=',', dtype=None)
            dense_w = np.reshape(dense_w, (128, 736))
            dense_w = np.transpose(dense_w, (1, 0))
            dense_b = genfromtxt(dirPath+'/dense_b.csv', delimiter=',', dtype=None)
            weights_dict[name] = [dense_w, dense_b]

    return weights_dict

In [9]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)
FRmodel.save('model.h5')

### Step4: Encoding Images 

In [10]:
def img_to_encoding1(image_path, model):
    img1 = cv2.imread(image_path, 1)
    dim = (96,96)
    img1 = cv2.resize(img1,dim, interpolation = cv2.INTER_AREA)
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

### Step5: Verifying model

In [38]:

def verify(image_path, identity, database, model):
    
    #encoding for the image.
    encoding = img_to_encoding1(image_path,model)
    # distance with identity'
    dist = np.linalg.norm(encoding - database[identity])

    if dist < 0.5:
        print("It's " + str(identity) + ", Welcome to Bank!!!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", Call the Police!!!")
        door_open = False
        
    return dist, door_open

### Using One-Shot Learning

In [49]:
database = {}
database["HJC_Peshchan"] = img_to_encoding1("genuine/NFI-00101001.png", FRmodel)
database["Vermututr"] = img_to_encoding1("genuine/NFI-00201002.png", FRmodel)
database["Yuan"] = img_to_encoding1("genuine/NFI-00301003.png", FRmodel)
database["Cbugn"] = img_to_encoding1("genuine/NFI-00401004.png", FRmodel)
database["GJ"] = img_to_encoding1("genuine/NFI-00501005.png", FRmodel)
database["Mary_Van_Camp"] = img_to_encoding1("genuine/NFI-00601006.png", FRmodel)
database["SUruin"] = img_to_encoding1("genuine/NFI-00701007.png", FRmodel)
database["Hmceycey"] = img_to_encoding1("genuine/NFI-00801008.png", FRmodel)
database["Pramod"] = img_to_encoding1("genuine/NFI-00901009.png", FRmodel)

database["Paul_J"] = img_to_encoding1("genuine/NFI-02303023.png", FRmodel)
database["Myuan_Priyush"] = img_to_encoding1("genuine/NFI-02404024.png", FRmodel)
database["Tanya"] = img_to_encoding1("genuine/NFI-02201022.png", FRmodel)

**Verify** for one shot learning.

<img src="test_images/Genuine.png"> Original Image

<img src="test_images/Forged.png"> Forged Image

In [39]:
verify("forged/NFI-00301001.png", "HJC_Peshchan", database, FRmodel)

It's not HJC_Peshchan, Call the Police!!!


(0.7450517, False)

<img src="test_images/Genuine1.png"> Genuine Image

In [40]:
verify("genuine/NFI-00104001.png", "HJC_Peshchan", database, FRmodel)

It's HJC_Peshchan, Welcome to Bank!!!


(0.27836695, True)

Able to detect subtle differences!!

<img src="test_images/YuanO.png"> Original Image

<img src="test_images/YuanF.png"> Forged Image

In [41]:
verify("forged/NFI-00405003.png", "Yuan", database, FRmodel)

It's Yuan, Welcome to Bank!!!


(0.47795907, True)

<img src="test_images/YuanG.png"> Genuine Image

In [42]:
verify("genuine/NFI-00302003.png", "Yuan", database, FRmodel)

It's Yuan, Welcome to Bank!!!


(0.46050698, True)

NOT Able to detect subtle differences!!

<img src="test_images/VermututrO.png"> Original Image

<img src="test_images/VermututrG.png"> Genuine Image

In [43]:
verify("genuine/NFI-00203002.png", "Vermututr", database, FRmodel)

It's Vermututr, Welcome to Bank!!!


(0.46900582, True)

<img src="test_images/VermututrF.png"> Forged Image

In [44]:
verify("forged/NFI-00302002.png", "Vermututr", database, FRmodel)

It's not Vermututr, Call the Police!!!


(0.70124537, False)

Able to detect subtle changes!!!

<img src="test_images/TanyaO.png"> Original Image

<img src="test_images/TanyaG.png"> Genuine Image

In [50]:
verify("genuine/NFI-02202022.png", "Tanya", database, FRmodel)

It's Tanya, Welcome to Bank!!!


(0.3526865, True)

<img src="test_images/TanyaF.png"> Forged Image

In [51]:
verify("forged/NFI-00501022.png", "Tanya", database, FRmodel)

It's not Tanya, Call the Police!!!


(0.5785195, False)

Able to detect subtle changes!!!

In [48]:
img_to_encoding1("genuine/NFI-02201022.png", FRmodel)

array([[ 9.19965357e-02,  1.07305624e-01, -2.56523583e-02,
        -6.04756884e-02,  1.18962966e-01,  7.69163743e-02,
         4.89022508e-02,  2.18805354e-02, -1.46911517e-01,
        -2.68563908e-02, -3.65193188e-02, -4.46496671e-03,
         7.35994279e-02,  5.92309702e-03,  1.83346550e-04,
         2.09649019e-02,  7.18601719e-02, -7.76038542e-02,
        -1.32264599e-01,  2.09488317e-01,  3.73670757e-02,
         2.90040709e-02,  2.95063127e-02,  6.18372462e-04,
        -6.43055588e-02,  2.64943484e-02, -1.14349715e-01,
        -2.62366012e-02,  7.50855282e-02,  3.50988982e-03,
        -9.75460559e-02,  6.04243912e-02, -1.03611805e-01,
         8.91832337e-02, -6.87223598e-02,  4.55054790e-02,
         1.57622918e-02,  2.22758129e-01, -8.24320912e-02,
         4.59416956e-03,  1.29053220e-02,  7.23915547e-02,
        -1.01118172e-02, -3.32452916e-02, -7.51554370e-02,
         1.64466947e-01,  1.18728481e-01,  2.08815727e-02,
        -1.02256209e-01,  8.74156505e-02,  5.53303659e-0

In [29]:
# img_to_encoding1("forged/NFI-00401003.png", FRmodel)

array([[ 0.1040557 ,  0.14639431, -0.07380492, -0.09834596,  0.04153858,
         0.11747582, -0.03230847,  0.04555031, -0.0156483 , -0.06735273,
        -0.01550347, -0.03578096,  0.02269405, -0.00298959, -0.03529763,
         0.01043521,  0.02132434, -0.13309786, -0.14070717,  0.15531158,
         0.04126226,  0.07089272,  0.03167374,  0.01084924, -0.10053892,
        -0.01308829, -0.02469324, -0.03853963,  0.01978707, -0.00459529,
        -0.08815479, -0.0243421 , -0.05690145,  0.07646212, -0.07781163,
         0.09322509,  0.03752657,  0.18610103, -0.11345245,  0.07004084,
        -0.06997257,  0.04245119, -0.09933359, -0.04784032,  0.01983559,
         0.1984514 ,  0.09372989,  0.01937401, -0.12868862,  0.16689777,
        -0.00308439,  0.14334612, -0.11759578,  0.08946506,  0.12416398,
        -0.08096603,  0.02606221,  0.01811345,  0.01859047, -0.13931578,
        -0.0040506 ,  0.05017831,  0.10614674, -0.09526715, -0.03956392,
        -0.15489021,  0.07712438,  0.19285089,  0.0

In [17]:
# img_to_encoding1("genuine/NFI-00103001.png", FRmodel)

array([[ 0.11717176,  0.04423836, -0.11427275, -0.03848497,  0.14286608,
         0.05382209,  0.06395762,  0.00444713,  0.00554071, -0.0871586 ,
         0.00803049,  0.05278345,  0.0861866 ,  0.09689719, -0.00292422,
        -0.08748735,  0.10283022, -0.10693506, -0.11507187,  0.20999338,
         0.02240222, -0.00395892,  0.07314583,  0.0421036 , -0.02757388,
        -0.07125108, -0.05095497, -0.01434501,  0.0224327 ,  0.03930177,
        -0.00660794, -0.01289282, -0.01572733,  0.04408295, -0.07684384,
         0.07413968,  0.00299342,  0.24544401, -0.14409228, -0.01436194,
         0.00410425,  0.13664557, -0.11182234, -0.00612561, -0.15001889,
         0.11832305,  0.01601231,  0.0328695 ,  0.01195212,  0.09618542,
        -0.06390809,  0.10124411, -0.07943651,  0.01180394,  0.127187  ,
        -0.07303889, -0.0930092 ,  0.13251705, -0.01785541, -0.09231742,
        -0.09570184, -0.0072075 ,  0.12259635, -0.13481224,  0.00625706,
        -0.0890718 ,  0.08139711,  0.19156003, -0.0

In [18]:
# a = img_to_encoding1("genuine/NFI-00102001.png", FRmodel)

In [19]:
# b = img_to_encoding1("genuine/NFI-00103001.png", FRmodel)

In [23]:
# a1 = np.array([[1,2]])
# b1 = np.array([[1,2]])

In [26]:
# (a+b)/2

array([[ 1.15688615e-01,  3.20313871e-02, -9.27501768e-02,
        -5.37407696e-02,  1.32184938e-01,  8.10448602e-02,
         6.96275905e-02,  3.45241986e-02, -3.45069170e-02,
        -8.24541599e-02, -1.30007956e-02,  5.49336709e-02,
         8.09830427e-02,  9.75692272e-02,  1.33590894e-02,
        -1.02557443e-01,  1.09500021e-01, -1.02464393e-01,
        -8.63782167e-02,  1.96165651e-01,  9.97928437e-03,
         1.34713855e-02,  7.86842629e-02,  5.81782982e-02,
        -4.74950634e-02, -7.36115873e-02, -2.86515281e-02,
        -3.21474671e-03,  1.35241505e-02,  2.07400415e-02,
        -2.81391088e-02, -5.59066832e-02, -1.63930133e-02,
         5.76668084e-02, -7.34250247e-02,  6.56741410e-02,
         6.78066211e-03,  2.17143476e-01, -1.77916348e-01,
        -1.66024528e-02,  7.67319556e-03,  1.33220896e-01,
        -9.18806568e-02,  2.86759040e-03, -1.66994482e-01,
         1.15551919e-01,  2.51331553e-02,  3.63404155e-02,
        -1.94187313e-02,  1.22499630e-01, -4.45030928e-0